In [97]:
import os
import yaml
import json
from typing import TypedDict, List, Optional

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langgraph.graph import StateGraph, END
from tavily import TavilyClient
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage

## 사용할 프롬프트 정의

load_dotenv()
tavily_client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

with open("translator.yaml", "r", encoding="utf-8") as f:
    prompt_config = yaml.safe_load(f)

GUIDELINES = prompt_config['alt_text_guidelines']

In [98]:
llm = ChatOpenAI(model="gpt-4o", temperature=0.3, streaming=True)

agent_llm = ChatOpenAI(model="gpt-4o", temperature=0.3)

tavily_tool = TavilySearchResults(max_results=3)
tools = [tavily_tool]

In [99]:
class GraphState(TypedDict):
    """그래프의 전체 상태를 정의"""
    # 입력
    original_alt_text: str
    nation: str
    image_type: str
    image_url: str
    
    # 생성 과정
    on_the_fly_guidelines: Optional[List[str]]
    generated_alt_text: Optional[str]
    
    # 평가 과정
    feedback: Optional[str]
    accessibility_score: Optional[int]
    cultural_score: Optional[int]
    
    # 에러 처리
    error: Optional[str]

# Guideline Agent의 출력을 위한 Pydantic 모델
class OnTheFlyGuidelines(BaseModel):
    on_the_fly_guidelines: List[str] = Field(description="A list of 2-3 specific, on-the-fly cultural guidelines.")

# Evaluator의 출력을 위한 Pydantic 모델
class Evaluation(BaseModel):
    accessibility_score: int = Field(description="The score for accessibility (1-5).")
    cultural_score: int = Field(description="The score for cultural appropriateness (1-5).")
    feedback: str = Field(description="If EITHER score is lower than 4, provide feedback. Otherwise, 'None'.")


In [100]:
# Guideline Agent 생성
guideline_agent_prompt = ChatPromptTemplate.from_messages([
    ("system", prompt_config['guideline_synthesizer']['system']),
    ("user", [
        {"type": "text", "text": prompt_config['guideline_synthesizer']['user']},
        {"type": "image_url", "image_url": {"url": "{image_url}"}}
    ]),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])
guideline_agent = create_openai_tools_agent(agent_llm, tools, guideline_agent_prompt)
guideline_agent_executor = AgentExecutor(agent=guideline_agent, tools=tools, verbose=True)

# Generator Chain 생성
generator_system_prompt = prompt_config['generator']['system'].format(alt_text_guidelines=GUIDELINES, on_the_fly_guidelines="{on_the_fly_guidelines}")
generator_user_prompt = prompt_config['generator']['user']
generation_prompt_template = ChatPromptTemplate.from_messages([
    ("system", generator_system_prompt),
    ("user", [
        {"type": "text", "text": generator_user_prompt},
        {"type": "image_url", "image_url": {"url": "{image_url}"}}
    ])
])
generation_chain = generation_prompt_template | llm

# Evaluator Chain 생성
evaluator_system_prompt = prompt_config['evaluator']['system'].format(alt_text_guidelines=GUIDELINES)
evaluator_prompt = ChatPromptTemplate.from_messages([
    ("system", evaluator_system_prompt),
    ("user", [
        {"type": "text", "text": prompt_config['evaluator']['user']},
        {"type": "image_url", "image_url": {"url": "{image_url}"}}
    ])
])
evaluation_chain = evaluator_prompt | agent_llm.with_structured_output(Evaluation)


/opt/anaconda3/envs/altauthor/lib/python3.12/site-packages/langchain_openai/chat_models/base.py:1771: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(


In [101]:
def guideline_agent_node(state: GraphState) -> GraphState:
    """Phase 1: 자율적인 에이전트가 이미지를 분석하고, 웹 검색을 통해 맞춤 가이드라인을 생성합니다."""
    print("--- 🕵️ EXECUTING GUIDELINE AGENT ---")
    try:
        agent_vars = {
            "nation": state["nation"],
            "original_alt_text": state["original_alt_text"],
            "image_url": state["image_url"],
        }
        result = guideline_agent_executor.invoke(agent_vars)
        
        # 에이전트의 출력에서 JSON 부분만 안전하게 추출
        json_str = result['output'][result['output'].find('{'):result['output'].rfind('}')+1]
        guideline_json = json.loads(json_str)
        
        # Pydantic으로 유효성 검사 및 데이터 추출
        on_the_fly_guidelines = OnTheFlyGuidelines(**guideline_json).on_the_fly_guidelines
        print(f"On-the-fly guidelines created: {on_the_fly_guidelines}")
        
        return {"on_the_fly_guidelines": on_the_fly_guidelines, "error": None}

    except Exception as e:
        print(f"An error occurred in the guideline agent: {e}")
        return {"error": str(e)}

def generation_node(state: GraphState) -> GraphState:
    """Phase 2: 생성된 가이드라인에 따라 최종 Alt Text를 생성합니다."""
    print("--- ✍️ EXECUTING GENERATOR ---")
    try:
        g_vars = {
            "on_the_fly_guidelines": "\n".join([f"- {g}" for g in state["on_the_fly_guidelines"]]),
            "nation": state["nation"],
            "image_type": state["image_type"],
            "image_url": state["image_url"],
            "original_alt_text": state["original_alt_text"],
            "previous_attempt": state.get("generated_alt_text", "N/A"),
            "feedback": state.get("feedback", "N/A")
        }

        # Vision 입력을 포함한 멀티모달 프롬프트 생성
        prompt_with_vision = generation_prompt_template.invoke(g_vars)
        final_alt_text = llm.invoke(prompt_with_vision).content
        print(f"Generated Alt-Text: {final_alt_text}")

        return {"generated_alt_text": final_alt_text, "error": None}

    except Exception as e:
        print(f"An error occurred in the generator: {e}")
        return {"error": str(e)}

def evaluator_node(state: GraphState) -> GraphState:
    """생성된 Alt Text를 Vision을 사용하여 평가합니다."""
    print("--- 🧐 EXECUTING EVALUATOR ---")
    try:
        e_vars = {
            "image_url": state["image_url"],
            "generated_alt_text": state["generated_alt_text"],
            "image_type": state["image_type"],
            "nation": state["nation"]
        }
        evaluation = evaluation_chain.invoke(e_vars)

        print(f"Evaluation Score (Accessibility): {evaluation.accessibility_score}")
        print(f"Evaluation Score (Cultural): {evaluation.cultural_score}")
        print(f"Evaluation Feedback: {evaluation.feedback}")

        return {
            "accessibility_score": evaluation.accessibility_score,
            "cultural_score": evaluation.cultural_score,
            "feedback": evaluation.feedback,
            "error": None
        }
    except Exception as e:
        print(f"An error occurred in the evaluator: {e}")
        return {"error": str(e)}

# --- 5. 조건부 엣지 및 그래프 조립 ---

def should_continue(state: GraphState) -> str:
    """평가 점수에 따라 다음 단계를 결정합니다."""
    print("--- 🤔 CHECKING THRESHOLD ---")
    if state.get("error"):
        print(f"--- 🛑 ERROR DETECTED: {state['error']}, ENDING GRAPH ---")
        return "end"
    
    if state.get("accessibility_score") is None or state.get("cultural_score") is None:
        print("--- ⚠️ Scores not found, ending graph to prevent loop. ---")
        return "end"

    if state["accessibility_score"] < 4 or state["cultural_score"] < 4:
        print(f"--- ❌ THRESHOLD FAILED (A:{state['accessibility_score']}, C:{state['cultural_score']}). LOOPING BACK ---")
        return "generation_node" # 피드백을 가지고 generation_node로 돌아감
    else:
        print(f"--- ✅ THRESHOLD PASSED (A:{state['accessibility_score']}, C:{state['cultural_score']}) ---")
        return "end"

workflow = StateGraph(GraphState)

# 3개의 노드 추가
workflow.add_node("guideline_agent", guideline_agent_node)
workflow.add_node("generation_node", generation_node)
workflow.add_node("evaluator", evaluator_node)

# 엣지 연결 (순서: guideline -> generation -> evaluation -> conditional)
workflow.set_entry_point("guideline_agent")
workflow.add_edge("guideline_agent", "generation_node")
workflow.add_edge("generation_node", "evaluator")
workflow.add_conditional_edges(
    "evaluator",
    should_continue,
    {"end": END, "generation_node": "generation_node"} # 실패 시 generation_node로
)

app = workflow.compile()


# --- 6. 그래프 실행 ---

IMAGE_URL = "https://biz.chosun.com/resizer/v2/PGCZYM62ST7D5X6JCERR6KJVQU.jpg?auth=19084ead847294a540268dc80e231db77a855c0b25a127a8ad640066dbb2b72c&width=464"

inputs = {
    "original_alt_text": "김장 중 김치를 먹고 있는 아이들의 모습.",
    "nation": "English",
    "image_type": "Photos and portraits",
    "image_url": IMAGE_URL
}

# 모든 이벤트를 저장할 리스트를 생성합니다.
events = []
# 루프의 마지막 상태를 저장할 딕셔너리를 생성합니다.
final_state = {}

print("\n--- 🚀 STARTING GRAPH EXECUTION ---")
# 'with' 구문을 제거하고 for 루프를 직접 사용합니다. 이것이 올바른 문법입니다.
for event in app.stream(inputs, {"recursion_limit": 50}):
    # 나중에 분석하기 위해 모든 이벤트를 저장합니다.
    events.append(event)
    for key, value in event.items():
        print(f"\nNode: {key}")
        # 루프가 끝나기 직전의 완전한 상태를 계속 업데이트하며 저장합니다.
        # '__end__' 키는 최종 상태가 아니므로 제외합니다.
        if key != "__end__":
            final_state.update(value)

print("\n\n--- ✨ FINAL RESULT ---")
# 루프가 모두 끝난 후, 저장된 final_state 딕셔너리에서 직접 값을 가져옵니다.

print(f"\nOn-the-fly Guidelines Created: {final_state.get('on_the_fly_guidelines')}")
print(f"Final Generated Alt-Text: {final_state.get('generated_alt_text')}")
print(f"Final Accessibility Score: {final_state.get('accessibility_score')}")
print(f"Final Cultural Score: {final_state.get('cultural_score')}")


--- 🚀 STARTING GRAPH EXECUTION ---
--- 🕵️ EXECUTING GUIDELINE AGENT ---


> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'Kimchi cultural significance Korea'}`


[{'title': 'Kimchi throughout millennia: a narrative review on the early and ...', 'url': 'https://journalofethnicfoods.biomedcentral.com/articles/10.1186/s42779-023-00171-w', 'content': 'common thing that cannot be found elsewhere. This could be the suggested reason behind the love and pride of Korean people for kimchi. According to a poll involving Korean people in 2006, kimchi was cited as the symbol of national culture representing Korea by 22.1% respondents, second only to the national flag of South Korea, _taegeukgi_ voted by 34.9% respondents, followed by _Hangul_, the Korean writing system (17.2%), _mugunghwa_, the national emblem-flower (13.9%) and _dogdo_, small islets [...] The importance of kimchi in Korean culture is reflected from a special annual event dedicated to